In [13]:
# measurement

import serial
import time

ser = serial.Serial('COM8', 9600)
time.sleep(2)

measurements = [] # 측정한 전체 데이터
current_data = [] # 측정할 때 사용할 딕셔너리
measurement_started = False
saving = False
try:
    while True:
        if ser.inWaiting() > 0:

            line = ser.readline().decode('utf-8').rstrip() # 데이터 읽기

            if "Arduino" in line:
                print(line)
            
            if "Stop" in line:
                print(line)
                ser.close()
                break
            
            if "Start measurement" in line:
                print(line)
                measurement_started = True
                current_measurement = {"time": time.time(), "pin1": [], "pin2": []}  # 새로운 측정을 위해 초기화
            elif "End measurement" in line:
                print(line)
                print(f"measurement for {time.time() - current_measurement['time']}")
                measurement_started = False
                current_measurement["time"] = time.time() - current_measurement["time"]  # 측정 시간 계산
                measurements.append(current_measurement)  # 현재 측정 데이터를 전체 리스트에 추가
                print(measurements)
            elif measurement_started:
                if line.startswith("pin1: "):
                    value = int(line.split(": ")[1])
                    current_measurement["pin1"].append(value)
                elif line.startswith("pin2: "):
                    value = int(line.split(": ")[1])
                    current_measurement["pin2"].append(value)
            
except KeyboardInterrupt:
    ser.close()
    print("Serial port closed.")
    print("Collected data:", measurements)
            

--------Arduino ON--------
Start measurement
End measurement. (0.90sec)
measurement for 1.2228014469146729
[{'time': 1.2228014469146729, 'pin1': [22, 15, 23, 6], 'pin2': [10, 16, 12, 12, 17, 19, 19, 28]}]
Start measurement
End measurement. (1.11sec)
measurement for 1.425116777420044
[{'time': 1.2228014469146729, 'pin1': [22, 15, 23, 6], 'pin2': [10, 16, 12, 12, 17, 19, 19, 28]}, {'time': 1.425116777420044, 'pin1': [6, 19, 6, 11], 'pin2': [12, 15, 10, 10, 8, 12, 17, 13, 15, 12, 18]}]
Start measurement
End measurement. (1.21sec)
measurement for 1.5260529518127441
[{'time': 1.2228014469146729, 'pin1': [22, 15, 23, 6], 'pin2': [10, 16, 12, 12, 17, 19, 19, 28]}, {'time': 1.425116777420044, 'pin1': [6, 19, 6, 11], 'pin2': [12, 15, 10, 10, 8, 12, 17, 13, 15, 12, 18]}, {'time': 1.5260529518127441, 'pin1': [8, 18, 8, 7], 'pin2': [11, 16, 9, 11, 10, 15, 17, 18, 23, 18, 18, 18]}]
Start measurement
End measurement. (1.11sec)
measurement for 1.4250812530517578
[{'time': 1.2228014469146729, 'pin1': 

In [12]:
# 예외 떴을 때 사용 (시리얼 통신 끊기)
ser.close()

In [14]:
# initializing CustomDataset
import torch
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from custom_dataset import CustomDataset
file_name = 'guttural' 

In [15]:
# save Dataset (기존 데이터셋이 있으면 ConcatDataset 인스턴스로 변경 후 데이터 추가 후 저장, 없으면 새 데이터셋(CustomDataset)을 생성)
new_dataSet = CustomDataset(measurements)

try:
    existing_dataSet = torch.load(file_name + '.pth')
    if isinstance(new_dataSet, CustomDataset):
        combined_dataSet = ConcatDataset([existing_dataSet, new_dataSet])
        print('기존에 데이터셋이 있으므로 기존 데이터셋에 데이터를 추가합니다.')
        torch.save(combined_dataSet, file_name + '.pth')
    else:
        combined_dataSet = new_dataSet
        print('기존에 데이터셋은 존재하나 CustomDataset이 아니므로 스킵합니다.')
except FileNotFoundError:
    combined_dataSet = new_dataSet
    print('기존에 데이터셋이 존재하지 않으므로 새로운 데이터셋을 추가합니다.')
    torch.save(combined_dataSet, file_name + '.pth')

기존에 데이터셋이 있으므로 기존 데이터셋에 데이터를 추가합니다.


In [16]:
# ConcatDataset -> CustomDataset으로 변환
from torch.utils.data.dataset import ConcatDataset
loaded_dataset = torch.load(file_name + '.pth')
print(loaded_dataset)
if isinstance(loaded_dataset, CustomDataset):
    print('현재 파일은 CustomDataset의 인스턴스입니다.')
else:
    print('현재 파일은 CustomDataset의 인스턴스가 아닙니다. CustomDataset으로 변환합니다.')
    # ConcatDataset에 포함된 모든 데이터셋의 요소를 모두 모아 하나의 리스트로 만듬
    all_data = []
    for data_tuple in loaded_dataset:
        data_point = {'time': data_tuple[0], 'pin1': data_tuple[1], 'pin2': data_tuple[2]}
        all_data.append(data_point)
    
    # 새로운 CustomDataset 객체 생성 및 저장
    custom_dataset = CustomDataset(all_data)
    torch.save(custom_dataset, file_name + '.pth')

현재 파일은 CustomDataset의 인스턴스가 아닙니다. CustomDataset으로 변환합니다.


In [17]:
# load Dataset
from torch.nn.utils.rnn import pad_sequence
from padding import PaddingCollate
loaded_dataset = torch.load(file_name + '.pth')

# DataLoader에 로드된 데이터셋 사용
dataloader = DataLoader(loaded_dataset, batch_size=100, shuffle=True, collate_fn=PaddingCollate())
for time, pin1, pin2 in dataloader:
    print(time.size())
    print("Time:", time)
    print("Pin1:", pin1.int())
    print("Pin2:", pin2.int())


torch.Size([50])
Time: tensor([1.3243, 1.4257, 1.7280, 1.7281, 1.3244, 1.7279, 1.7272, 1.5262, 1.4251,
        1.5261, 1.7271, 1.6270, 1.4252, 1.9310, 1.7281, 1.8286, 1.7271, 1.4243,
        1.9301, 1.3241, 1.4252, 1.8280, 1.4253, 1.7270, 1.6270, 1.7270, 1.4248,
        1.8285, 1.7278, 1.7272, 2.0311, 1.2228, 1.9291, 1.7276, 1.9306, 1.5250,
        1.9300, 1.7273, 1.7271, 1.3232, 1.4248, 1.8284, 1.4251, 1.4262, 1.8282,
        1.4251, 1.8281, 1.8281, 1.4258, 1.4252])
Pin1: tensor([[ 5, 17, 18, 17,  0],
        [33, 10,  0,  0,  0],
        [41, 26, 37,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 6, 12, 22,  0,  0],
        [ 8,  6, 19,  9,  0],
        [35,  0,  0,  0,  0],
        [28,  6,  7,  5,  0],
        [ 6, 10,  8,  8,  0],
        [ 8, 18,  8,  7,  0],
        [ 0,  0,  0,  0,  0],
        [32, 18, 19, 20,  5],
        [13,  0,  0,  0,  0],
        [15,  0,  0,  0,  0],
        [ 5,  5,  5,  7,  0],
        [13,  0,  0,  0,  0],
        [ 7,  0,  0,  0,  0],
        [ 8,